In [1]:
import geemap
import ee
import os
import pandas as pd

In [2]:
#ee.Authenticate()
# geemap.update_package()

In [3]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
##Loading CSVs
##PC
in_csv = 'C:/Users/arj26323/Documents/Data/Biomass datasets/BMA.csv'
df = geemap.csv_to_pandas(in_csv)
df = df [['Date', 'Longitude', 'Latitude']]

##Load feature class
##PC
df_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/BMA_fc.csv' ##Can only be done once file created; see below

bma_points = geemap.xy_to_points(df_fc, latitude = 'Latitude', longitude = 'Longitude')
Map.addLayer(bma_points, {}, 'BMA sampling locations')

In [5]:
print(df)
# print(df.dtypes)

      Date  Longitude   Latitude
0   29-Sep -81.289612  31.420226
1   29-Sep -81.289615  31.420161
2   29-Sep -81.289577  31.420091
3   29-Sep -81.289622  31.420043
4   29-Sep -81.289572  31.419946
..     ...        ...        ...
95  21-May -81.285426  31.390442
96  21-May -81.285451  31.390501
97  21-May -81.285515  31.390622
98  21-May -81.285523  31.390656
99  21-May -81.285584  31.390643

[100 rows x 3 columns]


In [6]:
##Load planetdata geotiff imagery
##PC
dean_may = r'E:/BMA planet data/21_May_2021/files/20210521_151835_36_2444_3B_AnalyticMS_SR_clip.tif'
dean_mar_a = r'E:/BMA planet data/22_Mar_2021/files/20210322_154607_103c_3B_AnalyticMS_SR_clip.tif'
dean_mar_b = r'E:/BMA planet data/22_Mar_2021/files/20210322_154608_103c_3B_AnalyticMS_SR_clip.tif'
dean_sep30_a = r'E:/BMA planet data/22_Mar_2021/files/20200930_154455_1011_3B_AnalyticMS_SR_clip.tif'
dean_sep30_b = r'E:/BMA planet data/22_Mar_2021/files/20200930_154456_1011_3B_AnalyticMS_SR_clip.tif'

dean_x = ee.Image(dean_may)

ls_x = r'C:/Users/arj26323/Downloads/landsat.tif'

Map.add_raster(dean_may, bands = [3, 2, 1], layer_name='May 2021') ##Not working-doesn't work if you manually load it either

Url()

In [7]:
dean_x

In [8]:
##Export CSV file - to then load as feature class
in_fc = df
out_dir = os.path.expanduser('~/Documents/Data/Biomass datasets')
out_csv = os.path.join(out_dir, 'BMA_fc.csv')
#df.to_csv('~/Documents/Data/Biomass datasets/BMA_fc.csv', index = False)
print(df)

      Date  Longitude   Latitude
0   29-Sep -81.289612  31.420226
1   29-Sep -81.289615  31.420161
2   29-Sep -81.289577  31.420091
3   29-Sep -81.289622  31.420043
4   29-Sep -81.289572  31.419946
..     ...        ...        ...
95  21-May -81.285426  31.390442
96  21-May -81.285451  31.390501
97  21-May -81.285515  31.390622
98  21-May -81.285523  31.390656
99  21-May -81.285584  31.390643

[100 rows x 3 columns]


In [17]:
##Organizing planet imagery for Acolite

import os, shutil

##PC path
path = (r"E:\BMA planet data\acolite_data\files\DN files")

os.chdir(path)

for f in os.listdir(path):
    folderName = f[:15]
    
    if not os.path.exists(folderName):
        os.mkdir(folderName)
        shutil.copy(os.path.join(path, f), folderName)
    else:
        shutil.copy(os.path.join(path, f), folderName)

In [18]:
##Bulk processing planet imagery for Acolite

indir = r'E:\BMA planet data\acolite_data\files\DN files' ##full path to your directory with files

output = r'E:\BMA planet data\acolite_data\acolite_output' ## path to put output files

path_to_acolite_mr = r'C:\Users\arj26323\Documents\acolite_mr' ## path to cloned github acolite_mr code

## import packages and acolite

import sys, os, glob

sys.path.append(path_to_acolite_mr)

import acolite as ac

## make a list of directories in the given indir path

files = glob.glob('{}/*/'.format(indir))

files.sort()

print('Found {} folders in {}.'.format(len(files), indir))

## additional keywords can be provided to acolite_mr_ac

limit=None ## process full scene, provide a 4 element list with [S, E, N, W] coordinates for processing a ROI

ancillary_data=False ## don't get ancillary MET and gas data

## run through each folder that was found

for file in files:
    print('Trying {}'.format(file))
    ac.acolite.acolite_mr_ac(file, output=output, ancillary_data=ancillary_data, limit=limit,export_geotiff=True)

Found 2 folders in E:\BMA planet data\acolite_data\files\DN files.
Trying E:\BMA planet data\acolite_data\files\DN files\20210521_151835\
20210521_151835_36_2444_3B_AnalyticMS_DN_clip.tif
20210521_151835_36_2444_3B_AnalyticMS_DN_metadata_clip.xml
20210521_151835_36_2444_3B_AnalyticMS_DN_udm_clip.tif
20210521_151835_36_2444_3B_udm2_clip.tif
Processing planet file
20210521_151835_36_2444_3B_AnalyticMS_DN_clip.tif
20210521_151835_36_2444_3B_AnalyticMS_DN_metadata_clip.xml
20210521_151835_36_2444_3B_AnalyticMS_DN_udm_clip.tif
20210521_151835_36_2444_3B_udm2_clip.tif


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [7]:
import rasterio
src = rasterio.open('E:/BMA planet data/21_May_2021/files/20210521_151835_36_2444_3B_AnalyticMS_SR_clip.tif')
src

<open DatasetReader name='E:/BMA planet data/21_May_2021/files/20210521_151835_36_2444_3B_AnalyticMS_SR_clip.tif' mode='r'>